In [1]:
import torch
import torch.nn as nn
from utils.datasets import get_testset
from utils.trained_models import get_trained_model

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
trained_model = get_trained_model().to(device)
test_loader = get_testset()

activations = {}

def relu_hook(module, input, output):
    print(f"Hook triggered for module: {module}")
    output = output.detach().cpu()
    total_neurons = output.numel()
    dead_neurons = (output == 0).sum().item()
    activations[module] = dead_neurons / total_neurons

# Register hooks for all ReLU layers
for name, layer in trained_model.named_modules():
    count = 0
    if isinstance(layer, nn.ReLU):
        print(++count)
        print(f"Registering hook for layer: {name}")
        layer.register_forward_hook(relu_hook)

# Pass a batch of data
trained_model.eval()
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        if inputs.dim() == 3:  # If 3D, add batch dimension
            inputs = inputs.unsqueeze(0)

        outputs = trained_model(inputs)
        break

# Display
if activations:
    for layer, percentage in activations.items():
        print(f"Layer: {layer}, Dying ReLU Percentage: {percentage:.3%}")
else:
    print("No hooks were triggered or no ReLU layers were found.")

C:\Users\Tianyi Z\Desktop\school work\CSC277\HW1\hw1_venv\problem_2\utils\trained_models.py:82: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.loa

Files already downloaded and verified
No hooks were triggered or no ReLU layers were found.
